## Bibliotecas

In [1]:
import pandas as pd

## Estratificação

In [2]:
df = pd.read_csv('../data/sp_sao_paulo.csv', low_memory=False)
df = df[['DATA_DO_PEDIDO','RAZAO_SOCIAL','DESCRICAO_DO_PRODUTO','SUB_BU','PRECO','QUANTIDADE_FATURADA']]
df['DATA_DO_PEDIDO'] = pd.to_datetime(df['DATA_DO_PEDIDO'], dayfirst=True).dt.strftime('%Y-%m')
df = df.query('PRECO > 0.0')
df.dropna()
df.head()


,DATA_DO_PEDIDO,RAZAO_SOCIAL,DESCRICAO_DO_PRODUTO,SUB_BU,PRECO,QUANTIDADE_FATURADA
0,2020-11,DROGAMELLO DROGARIA,CENTROTABS CABELOS E UNHAS CT FR 60 CAP,SMART,8.49,60
1,2020-11,NOVA MELLO DROGARIA,NEOLEFRIN XPE FR VD AMB 60ML+COPO MED,SMART,5.13,12
2,2020-11,NOVA MELLO DROGARIA,COLIRIO NEO BRASIL SOL OFT FR C/20ML,SMART,4.58,12
3,2020-11,NOVA MELLO DROGARIA,CIPROFIBRATO 100MG CT BL 2X15 COMP,GENÉRICOS,10.48,12
4,2020-11,NOVA MELLO DROGARIA,TORSILAX CT BL 2X6 COMP,SMART,2.43,140


In [3]:
#{ 'PRECO':['mean'] , 'QUANTIDADE_FATURADA':['sum']}
df_g = df.groupby(['DATA_DO_PEDIDO','RAZAO_SOCIAL','DESCRICAO_DO_PRODUTO','SUB_BU'],as_index=False).agg(PRECO_MEDIO=('PRECO','mean'),QUANTIDADE_TOTAL = ('QUANTIDADE_FATURADA','sum'))
df_g


,DATA_DO_PEDIDO,RAZAO_SOCIAL,DESCRICAO_DO_PRODUTO,SUB_BU,PRECO_MEDIO,QUANTIDADE_TOTAL
0,2020-11,A.R. NETO DROGARIAS LTDA - ME,CIPROFIBRATO 100MG CT BL 2X15 COMP,GENÉRICOS,10.48,10
1,2020-11,A.R. NETO DROGARIAS LTDA - ME,DIPIRONA 500MG CAFEINA 65MG CT 2X8 COMP,GENÉRICOS,8.02,6
2,2020-11,A.R. NETO DROGARIAS LTDA - ME,DRAMAVIT B6 SOL OR CT FR PET 20 ML LP,SMART,4.16,3
3,2020-11,A.R. NETO DROGARIAS LTDA - ME,EQUILIBRISSE CT BL AL 420MG X 10,SMART,6.20,6
4,2020-11,A.R. NETO DROGARIAS LTDA - ME,GLIBENCLAMIDA 5MG CT BL 30 COMP,GENÉRICOS,1.05,5
...,...,...,...,...,...,...
46666,2021-12,ZLZ DROG COM VAREJISTA DE PROD FARMACS E,HISTAMIN CREME 10MG/G BG C/30G,SMART,5.94,3
46667,2021-12,ZLZ DROG COM VAREJISTA DE PROD FARMACS E,NEOLEFRIN CT BL 20 COMP,SMART,2.76,8
46668,2021-12,ZLZ DROG COM VAREJISTA DE PROD FARMACS E,NEOLEFRIN DIA CT BL 20 COMP,SMART,2.30,4
46669,2021-12,ZLZ DROG COM VAREJISTA DE PROD FARMACS E,NEOLEFRIN XPE FR VD AMB 60ML+COPO MED,SMART,4.80,2


## Modelagem

In [4]:
# split data into train-test set
train = df_g[df_g['DATA_DO_PEDIDO'] < '2021-08']
test =  df_g[df_g['DATA_DO_PEDIDO'] >= '2021-08']


In [5]:
# import the regression module
from pycaret.regression import *
# initialize setup
s = setup(data=train,
          preprocess=True,
          test_data=test,
          target='QUANTIDADE_TOTAL',
          fold_strategy='timeseries',
          fold=3, 
          transform_target=True,
          remove_outliers = True,
          pca=False,
          pca_method ="kernel",
          session_id=123)


,Description,Value
0,session_id,123
1,Target,QUANTIDADE_TOTAL
2,Original Data,"(30528, 6)"
3,Missing Values,False
4,Numeric Features,1
5,Categorical Features,3
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(29001, 449)"


In [6]:
best = compare_models(sort = 'RMSE')

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,53.6900,251687.2598,484.9996,0.5763,0.8524,1.0233,19.3967
rf,Random Forest Regressor,56.5225,382645.2891,562.3022,0.4827,0.8589,0.9806,12.1967
gbr,Gradient Boosting Regressor,65.8081,588194.5243,730.9947,0.1610,0.9893,1.0245,3.0400
knn,K Neighbors Regressor,64.3562,602701.6327,732.5403,0.1611,0.9808,1.1530,3.8233
lightgbm,Light Gradient Boosting Machine,62.0747,661993.1412,770.6909,0.0843,0.8826,0.9626,0.2200
dt,Decision Tree Regressor,71.4001,741330.3054,778.1582,0.1128,1.0416,1.6677,0.9100
ada,AdaBoost Regressor,75.0080,705220.7480,798.9637,0.0029,1.3044,1.0913,4.4800
en,Elastic Net,76.5004,712184.4572,803.2391,-0.0087,1.3929,1.4127,0.4067
lasso,Lasso Regression,76.5382,712233.7020,803.2715,-0.0088,1.3986,1.4148,0.5967
llar,Lasso Least Angle Regression,76.8496,712418.2810,803.3943,-0.0092,1.4347,1.4937,0.5933


## Controle

In [7]:
controle = df_g.query('DESCRICAO_DO_PRODUTO == "NEOLEFRIN XPE FR VD AMB 60ML+COPO MED"')
# create 12 month moving average
controle['MA12'] = controle['QUANTIDADE_TOTAL'].rolling(12).mean()
# plot the controle and MA
import plotly.express as px
fig = px.line(controle, x="DATA_DO_PEDIDO", y=["QUANTIDADE_TOTAL", "MA12"], template = 'plotly_dark')
fig.show()

In [8]:
# generate predictions on the original dataset
predictions = predict_model(best, data=controle)
# add a date column in the dataset
# predictions['DATA'] = pd.date_range(start='2021-01-01', end = '2021-08-01', freq = 'MS')
# line plot
fig = px.line(predictions, x='DATA_DO_PEDIDO', y=["QUANTIDADE_TOTAL", "Label"], template = 'plotly_dark')
# add a vertical rectange for test-set separation
fig.add_vrect(x0="2021-01-01", x1='2021-08-01', fillcolor="grey", opacity=0.25, line_width=0)
fig.show()



,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Extra Trees Regressor,4.9262,448.5469,21.1789,0.8343,0.5233,0.2310


In [51]:
future_dates = pd.date_range(start = '01/2022', end = '01/2022', freq = 'MS')
future_df = pd.DataFrame()

for preco in range(1,10):

    future_df['DATA_DO_PEDIDO'] = future_dates
    future_df['DATA_DO_PEDIDO'] = pd.to_datetime(future_df['DATA_DO_PEDIDO'], dayfirst=True).dt.strftime('%Y-%m')
    future_df['RAZAO_SOCIAL']='UNIAO DROGARIAS'
    future_df['DESCRICAO_DO_PRODUTO'] = "NEOLEFRIN XPE FR VD AMB 60ML+COPO MED"
    future_df['SUB_BU'] = 'SMART'
    future_df["PRECO_MEDIO"] = preco

    predictions_future = predict_model(best, data=future_df)
    print(preco,predictions_future["Label"][0])

1 11.822568501770293
2 10.44966493832189
3 17.016229453237113
4 13.864580137566863
5 9.810523709996355
6 8.90668084816775
7 8.331529991980457
8 7.021364627154622
9 6.069203120762938


In [34]:
controle.query("RAZAO_SOCIAL == 'UNIAO DROGARIAS' ")

,DATA_DO_PEDIDO,RAZAO_SOCIAL,DESCRICAO_DO_PRODUTO,SUB_BU,PRECO_MEDIO,QUANTIDADE_TOTAL,MA12
19620,2021-04,UNIAO DROGARIAS,NEOLEFRIN XPE FR VD AMB 60ML+COPO MED,SMART,4.68,12,10.333333
26873,2021-06,UNIAO DROGARIAS,NEOLEFRIN XPE FR VD AMB 60ML+COPO MED,SMART,5.05,15,26.916667
46657,2021-12,UNIAO DROGARIAS,NEOLEFRIN XPE FR VD AMB 60ML+COPO MED,SMART,4.96,12,19.500000
